In [ ]:
!pip install polars

In [1]:
import pandas as pd
import lightgbm as lgb
from lightgbm import LGBMClassifier, Dataset, cv, train, early_stopping
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import __version__ as sklearn_version
from packaging import version
from multiprocessing import cpu_count
import numpy as np
import shap
import matplotlib.pyplot as plt
import polars as pl
import itertools
import seaborn as sns

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Feature preprocessing 

In [ ]:
### data preparation

try:
    comorbidities = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_comorbidity.csv')
except FileNotFoundError:
    print("Incorrect path to comorbidities file or comorbidities dataframe manipulation section not run first.")

try:
    adi_scores = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_adi_scores.csv')
except FileNotFoundError:
    print("Incorrect path to ADI scores file.")
try:
    labs = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_labs.csv')
except FileNotFoundError:
    print("Incorrect path to labs file.")
try:
    vitals = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_vitals.csv')
except FileNotFoundError:
    print("Incorrect path to vitals file.")
try:
    prior_infecting_organisms = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_culture_prior_infecting_organism.csv')
except FileNotFoundError:
    print("Incorrect path to prior infecting organisms file.")
try:
    subtype_exposure = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_antibiotic_subtype_exposure.csv')
except FileNotFoundError:
    print("Incorrect path to subtype exposure file.")
try:
    demographics = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_demographics.csv')
except FileNotFoundError:
    print("Incorrect path to demographics file.")

try:
    prior_med = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_prior_med.csv')
except FileNotFoundError:
    print("Incorrect path to prior medications file.")
try:
    prior_procedures = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_priorprocedures.csv')
except FileNotFoundError:
    print("Incorrect path to prior procedures file.")
try:
    ward_info = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_ward_info.csv')
except FileNotFoundError:
    print("Incorrect path to ward info file.")
try:
    microbial_resistance = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_microbial_resistance.csv')
except FileNotFoundError:
    print("Incorrect path to microbial resistance file.")

try:
    cohort = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_cohort.csv')
except FileNotFoundError:
    print("Incorrect path to cohort file.")

try:
    antibiotic_class_exposure = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_antibiotic_class_exposure.csv')
except FileNotFoundError:
    print("Incorrect path to antibiotic class exposure file.")
try:
    implied_susceptibility = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_implied_susceptibility.csv')
except FileNotFoundError:
   print("Incorrect path to implied susceptibility file.")
try:
    nursing_home_visits = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_nursing_home_visits.csv')
except FileNotFoundError:
    print("Incorrect path to nursing home visits file.")

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_89890/3956472702.py:9: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  adi_scores = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_adi_scores.csv')
/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_89890/3956472702.py:13: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  labs = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_labs.csv')
/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_89890/3956472702.py:17: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  vitals = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_vitals.csv')
/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_89890/3956472702.py:29: DtypeWarning: Columns (4) 

# only in patients with posetive culture

In [3]:
cohort = cohort[cohort['was_positive'] == 1]
cohort = cohort[cohort['ordering_mode'] == 'Inpatient']
cohort = cohort.drop(columns = ['ordering_mode', 'was_positive'])
cohort['year'] = [time[:4] for time in cohort['order_time_jittered_utc']]
cohort=cohort[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','order_time_jittered_utc','culture_description','year']]
cohort.drop_duplicates(inplace=True)

## implied_susceptibility needs to be filtered to only include cultures with organisms and antibiotics

In [4]:
implied_susceptibility=implied_susceptibility[(implied_susceptibility.organism!='Null')&(implied_susceptibility.antibiotic!='Null')]
implied_susceptibility.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,organism,antibiotic,susceptibility,implied_susceptibility
0,JC2673551,131337997983,831009441,ENTEROBACTER CLOACAE COMPLEX,imipenem,Null,Susceptible
1,JC616267,131007285560,358749950,ENTEROBACTER CLOACAE,imipenem,Null,Null
2,JC517471,131024758159,420255585,ENTEROBACTER ASBURIAE,doripenem,Null,Null
3,JC1931399,131200817434,512398445,MYCOBACTERIUM AVIUM COMPLEX,Clarithromycin,Susceptible,Null
4,JC605042,131280233261,642349435,KLEBSIELLA OXYTOCA,Ceftolozane/Tazobactam,Susceptible,Null


## implied_susceptibility needs to be filtered to only include cultures with susceptibility or implied_susceptibility

In [5]:

implied_susceptibility=implied_susceptibility[(implied_susceptibility.susceptibility!='Null')|(implied_susceptibility.implied_susceptibility!='Null')]

##  impute implied_susceptibility with first susceptibility and then clean to have only 2 categories susceptible and resistant

In [7]:
implied_susceptibility.loc[
    implied_susceptibility['implied_susceptibility'] == 'Null', 
    'implied_susceptibility'
] = implied_susceptibility['susceptibility']
implied_susceptibility=implied_susceptibility[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','organism','antibiotic','implied_susceptibility']]
implied_susceptibility=implied_susceptibility[implied_susceptibility.implied_susceptibility.isin(['Susceptible', 'Resistant','Non Susceptible'])]
implied_susceptibility.loc[implied_susceptibility.implied_susceptibility=='Non Susceptible',
                           'implied_susceptibility'
                           ]='Resistant'
implied_susceptibility.implied_susceptibility.unique()

array(['Susceptible', 'Resistant'], dtype=object)

In [8]:
df = cohort.merge(implied_susceptibility, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')
df = df[df['implied_susceptibility'].isin(['Susceptible', 'Resistant'])]
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant


## categorize the age and make it integer , fix gender type to be boolean

In [9]:
df = df.merge(demographics, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')

df.loc[df.age=='Null','age']=None
df.loc[df.gender=='Null','gender']=None
df.loc[df.gender=='0','gender']=0
df.loc[df.gender=='1','gender']=1

df.loc[df.age=='18–24 years','age']=1
df.loc[df.age=='25–34 years','age']=2
df.loc[df.age=='35–44 years','age']=3
df.loc[df.age=='45-54 years','age']=4
df.loc[df.age=='55-64 years','age']=5
df.loc[df.age=='65-74 years','age']=6
df.loc[df.age=='75-84 years','age']=7
df.loc[df.age=='85-89 years','age']=8
df.loc[df.age=='above 90','age']=9
df['age']=df['age'].astype(int)
df.age.unique()

array([7, 3, 6, 5, 4, 9, 8, 2, 1])

## Add the ward info to the cohort

In [10]:
ward_info=ward_info[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','hosp_ward_IP','hosp_ward_OP','hosp_ward_ER','hosp_ward_ICU']]
ward_info.drop_duplicates(inplace=True)
for col in ward_info.columns.values:
    ward_info.loc[ward_info[col]=='Null',col]=None
ward_info.hosp_ward_IP.unique()

array([0., 1.])

## Add ADI score

In [11]:
adi_scores=adi_scores[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','adi_score']]
adi_scores.drop_duplicates(inplace=True)
for col in adi_scores.columns.values:
    adi_scores.loc[adi_scores[col]=='Null',col]=None

In [12]:
df = df.merge(ward_info, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')
df = df.merge(adi_scores, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,gender,hosp_ward_IP,hosp_ward_OP,hosp_ward_ER,hosp_ward_ICU,adi_score
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,0,1.0,0.0,1.0,0.0,2
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,0,1.0,0.0,1.0,0.0,2
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,0,1.0,0.0,1.0,0.0,2
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,0,1.0,0.0,1.0,0.0,2
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,0,1.0,0.0,1.0,0.0,2


## fix vitals type data and add to cohort

In [13]:
for col in vitals.columns.values:
    if not col in ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']:
        vitals.loc[vitals[col]=='Null',col]=None
        vitals[col]=vitals[col].astype('float')
vitals.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,Q25_heartrate,Q75_heartrate,median_heartrate,Q25_resprate,Q75_resprate,median_resprate,Q25_temp,...,first_diasbp,last_diasbp,last_sysbp,first_sysbp,last_temp,first_temp,last_resprate,first_resprate,last_heartrate,first_heartrate
0,JC2361817,131308278530,718222807,86.0,86.0,86.0,NaN,NaN,NaN,NaN,...,NaN,80.0,109.0,NaN,NaN,NaN,NaN,NaN,NaN,86.0
1,JC2219930,131318072067,748261410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,75.0,75.0,121.0,121.0,NaN,NaN,NaN,NaN,NaN,NaN
2,JC2455425,131256420868,574712812,70.0,70.0,70.0,NaN,NaN,NaN,NaN,...,69.0,NaN,NaN,106.0,NaN,NaN,NaN,NaN,70.0,NaN
3,JC1826078,131213257732,514463599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,90.0,90.0,120.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,JC1541734,131021676805,410407692,39.0,78.0,73.0,NaN,NaN,NaN,NaN,...,NaN,42.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN,80.0


In [14]:
df = df.merge(vitals, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')

## Add nursing home visits

In [15]:
nursing_home_visits=nursing_home_visits[nursing_home_visits.nursing_home_visit_culture>0]
nursing_home_visits['nursing_home_visits_within6month']=nursing_home_visits['nursing_home_visit_culture'].apply(lambda x: 1 if  x <= 180 else 0)
nursing_home_visits=nursing_home_visits[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','nursing_home_visits_within6month']]
nursing_home_visits = nursing_home_visits.groupby(['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']).agg(
    nursing_visits_within_6mo =('nursing_home_visits_within6month', sum),
).reset_index()
nursing_home_visits['nursing_visits_within_6mo']=nursing_home_visits['nursing_visits_within_6mo'].astype(int)

df = df.merge(nursing_home_visits, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.head()

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_89890/2613113203.py:4: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  nursing_home_visits = nursing_home_visits.groupby(['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']).agg(


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,last_diasbp,last_sysbp,first_sysbp,last_temp,first_temp,last_resprate,first_resprate,last_heartrate,first_heartrate,nursing_visits_within_6mo
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN


## Add prior procedure for the past 6 month

In [16]:
prior_procedures=prior_procedures[(prior_procedures.procedure_description!='Null')&(prior_procedures.procedure_description.notna())]
prior_procedures=prior_procedures[(prior_procedures.procedure_time_to_culturetime.notna())&(prior_procedures.procedure_time_to_culturetime!='Null')]
prior_procedures=prior_procedures[(prior_procedures.procedure_time_to_culturetime>0)&(prior_procedures.procedure_time_to_culturetime<=180)]
procedures = prior_procedures.procedure_description.unique()

for procedure in procedures:
    prior_procedures[f"{procedure}_within_6mo"] = prior_procedures[prior_procedures['procedure_description'] == procedure]['procedure_time_to_culturetime'].apply(lambda x: 1 if x <= 180 else 0)
    

columns_to_sum = ['urethral_catheter_within_6mo','surgical_procedure_within_6mo','mechvent_within_6mo','cvc_within_6mo','parenteral_nutrition_within_6mo','dialysis_within_6mo']
agg_dict = {col: (col, 'sum') for col in columns_to_sum}
prior_procedures = prior_procedures.groupby(['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']).agg(**agg_dict).reset_index() 
prior_procedures=prior_procedures[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','urethral_catheter_within_6mo','surgical_procedure_within_6mo','mechvent_within_6mo','cvc_within_6mo','parenteral_nutrition_within_6mo','dialysis_within_6mo']]
prior_procedures.drop_duplicates(inplace=True)
for col in ['urethral_catheter_within_6mo','surgical_procedure_within_6mo','mechvent_within_6mo','cvc_within_6mo','parenteral_nutrition_within_6mo','dialysis_within_6mo']:
    prior_procedures[col]=prior_procedures[col].astype(int)

In [17]:
df = df.merge(prior_procedures, on =['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.iloc[:,42:]=df.iloc[:,42:].fillna(0)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,first_resprate,last_heartrate,first_heartrate,nursing_visits_within_6mo,urethral_catheter_within_6mo,surgical_procedure_within_6mo,mechvent_within_6mo,cvc_within_6mo,parenteral_nutrition_within_6mo,dialysis_within_6mo
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


## Add microbial_resistance over the past 6 months

In [18]:
microbial_resistance=microbial_resistance[(microbial_resistance.organism!='Null')&(microbial_resistance.organism.notna())]
microbial_resistance=microbial_resistance[(microbial_resistance.antibiotic!='Null')&(microbial_resistance.antibiotic.notna())]
microbial_resistance=microbial_resistance[(microbial_resistance.resistant_time_to_culturetime>0)&(microbial_resistance.resistant_time_to_culturetime<=180)]
microbial_resistance=microbial_resistance[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','organism','antibiotic']]


# filter for top 10 most common microbes
microbial_resistance = microbial_resistance[microbial_resistance['organism'].isin(['ESCHERICHIA COLI', 'PSEUDOMONAS AERUGINOSA', 'MUCOID PSEUDOMONAS AERUGINOSA', 
                                                                                   'KLEBSIELLA PNEUMONIAE', 'ACHROMOBACTER XYLOSOXIDANS', 'STAPHYLOCOCCUS AUREUS', 
                                                                                   'PSEUDOMONAS AERUGINOSA (NON-MUCOID CF)', 'ENTEROCOCCUS SPECIES', 'ENTEROBACTER CLOACAE COMPLEX', 'PROTEUS MIRABILIS'])]

microbial_resistance.drop_duplicates(inplace=True)
microbial_resistance['organism_resistance_antibiotic'] = microbial_resistance['organism'] + '_' + microbial_resistance['antibiotic']
microbial_resistance=microbial_resistance[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','organism_resistance_antibiotic']]
microbial_resistance.drop_duplicates(inplace=True)

In [19]:
microbial_resistance = microbial_resistance.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "organism_resistance_antibiotic"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
microbial_resistance.columns.name = None 

### for prior procedures null values imputed with zero

In [20]:
df = df.merge(microbial_resistance, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.iloc[:, 48:] = df.iloc[:, 48:].fillna(0)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,STAPHYLOCOCCUS AUREUS_Cefazolin,STAPHYLOCOCCUS AUREUS_Ciprofloxacin,STAPHYLOCOCCUS AUREUS_Erythromycin,STAPHYLOCOCCUS AUREUS_Gentamicin,STAPHYLOCOCCUS AUREUS_Levofloxacin,STAPHYLOCOCCUS AUREUS_Linezolid,STAPHYLOCOCCUS AUREUS_Moxifloxacin,STAPHYLOCOCCUS AUREUS_Penicillin,STAPHYLOCOCCUS AUREUS_Rifampin,STAPHYLOCOCCUS AUREUS_Vancomycin
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## add prior infecting organisms within 6 months

In [21]:
prior_infecting_organisms=prior_infecting_organisms[(prior_infecting_organisms.prior_infecting_organism_days_to_culutre<=180)&
                                                    (prior_infecting_organisms.prior_infecting_organism_days_to_culutre>0)]

prior_infecting_organisms=prior_infecting_organisms[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','prior_organism']]
prior_infecting_organisms.drop_duplicates(inplace=True)

In [22]:
prior_infecting_organisms = prior_infecting_organisms.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "prior_organism"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
prior_infecting_organisms.columns.name = None  
prior_infecting_organisms.columns = [f"prior_infected_{col}" if 3 <= i <= 19 else col for i, col in enumerate(prior_infecting_organisms.columns)]
prior_infecting_organisms = prior_infecting_organisms[prior_infecting_organisms.iloc[:, 3:20].gt(0).any(axis=1)]

In [23]:
df = df.merge(prior_infecting_organisms, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.iloc[:, 79:] = df.iloc[:, 79:].fillna(0)
df.drop_duplicates(inplace=True)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,prior_infected_Escherichia,prior_infected_Klebsiella,prior_infected_Morganella,prior_infected_Proteus,prior_infected_Providencia,prior_infected_Pseudomonas,prior_infected_Serratia,prior_infected_Staphylococcus,prior_infected_Stenotrophomonas,prior_infected_Streptococcus
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Add prior medication within last 6 month

In [24]:
prior_med=prior_med[(prior_med.medication_time_to_culturetime>0)&(prior_med.medication_time_to_culturetime<=180)]
prior_med=prior_med[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','medication_name']]
prior_med.drop_duplicates(inplace=True)

In [25]:
prior_med = prior_med.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "medication_name"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
prior_med.columns.name = None 
prior_med.columns = [f"prior_med_{col}" if 3 <= i <= 23 else col for i, col in enumerate(prior_med.columns)]
prior_med = prior_med[prior_med.iloc[:, 3:23].gt(0).any(axis=1)]

In [26]:
df = df.merge(prior_med, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'],how='left')
df.iloc[:, 95:] = df.iloc[:, 95:].fillna(0)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,prior_med_Colistin,prior_med_Ertapenem,prior_med_Gentamicin,prior_med_Isoniazid,prior_med_Levaquin,prior_med_Levofloxacin,prior_med_Linezolid,prior_med_Metronidazole,prior_med_Penicillin,prior_med_Vancomycin
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Add antibiotic_class_exposure

In [27]:
antibiotic_class_exposure=antibiotic_class_exposure[(antibiotic_class_exposure.time_to_culturetime>0)&(antibiotic_class_exposure.time_to_culturetime<=180)]
antibiotic_class_exposure=antibiotic_class_exposure[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','antibiotic_class']]
antibiotic_class_exposure.drop_duplicates(inplace=True)

In [ ]:
antibiotic_class_exposure = antibiotic_class_exposure.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "antibiotic_class"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
antibiotic_class_exposure.columns.name = None  
# fix the naming of the columns to include 'prior_abx_class_' prefix for the antibiotic class columns
antibiotic_class_exposure.columns = [f"prior_abx_class_{col}" if 3 <= i else col for i, col in enumerate(antibiotic_class_exposure.columns)]

In [29]:
df = df.merge(antibiotic_class_exposure, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.iloc[:, 115:] = df.iloc[:, 115:].fillna(0)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,prior_abx_class_Glycopeptide,prior_abx_class_Macrolide Lincosamide,prior_abx_class_Monobactam,prior_abx_class_Nitrofuran,prior_abx_class_Nitroimidazole,prior_abx_class_Oxazolidinone,"prior_abx_class_Polymyxin, Lipopeptide",prior_abx_class_Sulfonamide,prior_abx_class_Tetracycline,prior_abx_class_Urinary Antiseptic
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Add Labratory measurment 

In [ ]:
df = df.merge(labs, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')
df.head()

## For the real-time deployment, we will not use the prior comorbidity as it is not available in the real-time data and removing it from the dataframe wont affect the model performance significantly.

In [36]:
df.to_csv('Model1_AllABX_fixedTime0.csv',index=False)

## For the reaserch purpose we add comorbidities to the cohort as well

In [ ]:
comorbidities=comorbidities[((comorbidities.comorbidity_component_start_days_culture<=180) & (comorbidities.comorbidity_component_start_days_culture>=0)) &
                            ((comorbidities.comorbidity_component_end_days_culture<0) | (comorbidities.comorbidity_component_end_days_culture.isna()))]


comorbidities=comorbidities[(comorbidities.comorbidity_component!='Null')&(comorbidities.comorbidity_component.notna())]
comorbidities=comorbidities[["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "comorbidity_component"]]
comorbidities.drop_duplicates(inplace=True)

comorbidities = comorbidities.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "comorbidity_component"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
comorbidities.columns.name = None 
print(comorbidities)

In [ ]:
df = df.merge(comorbidities, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.iloc[:, 189:] = df.iloc[:, 189:].fillna(0)


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.to_csv('Model1_with_Comorbidity_components.csv',index=False)
df=pd.read_csv('Model1_with_Comorbidity_components.csv')

In [8]:
import numpy as np
from scipy.stats import entropy

from scipy.stats import entropy

def compute_kl_for_categorical(train_col, test_col):
    # Get frequency counts
    train_dist = train_col.value_counts(normalize=True)
    test_dist = test_col.value_counts(normalize=True)

    # Union of all categories
    all_cats = set(train_dist.index).union(set(test_dist.index))

    # Reindex and fill missing with 0
    train_probs = train_dist.reindex(all_cats, fill_value=0) + 1e-10
    test_probs = test_dist.reindex(all_cats, fill_value=0) + 1e-10

    # Normalize again after smoothing
    train_probs /= train_probs.sum()
    test_probs /= test_probs.sum()

    return entropy(train_probs, test_probs)

def compute_kl_divergence(train_col, test_col, bins=20):
    """
    Estimate KL divergence between train and test for a single feature.
    KL(train || test) = sum(P * log(P / Q))
    """
    # Remove NaNs
    train_col = train_col[~np.isnan(train_col)]
    test_col = test_col[~np.isnan(test_col)]

    # Use same binning for both
    min_val = min(train_col.min(), test_col.min())
    max_val = max(train_col.max(), test_col.max())
    bins = np.linspace(min_val, max_val, bins)

    train_hist, _ = np.histogram(train_col, bins=bins, density=True)
    test_hist, _ = np.histogram(test_col, bins=bins, density=True)

    # Smooth to avoid zero probabilities
    train_hist += 1e-10
    test_hist += 1e-10

    # Normalize to create probability distributions
    train_hist /= train_hist.sum()
    test_hist /= test_hist.sum()

    return entropy(train_hist, test_hist)  # KL divergence: D_KL(P || Q)

In [9]:
kl_results = {}

for col in train_data.columns:
    if pd.api.types.is_numeric_dtype(train_data[col]):
        print(f"Processing column: {col}")
        try:
            kl_value = compute_kl_divergence(train_data[col], test_data[col])
            kl_results[col] = kl_value
            print(f"KL divergence for {col}: {kl_value}")
        except Exception as e:
            kl_results[col] = f"Error: {e}"
    print("************************")
# Show results
kl_df = pd.DataFrame.from_dict(kl_results, orient='index', columns=['KL_Divergence']).sort_values('KL_Divergence', ascending=False)


************************
Processing column: pat_enc_csn_id_coded
KL divergence for pat_enc_csn_id_coded: 0.00023302176051164143
************************
Processing column: order_proc_id_coded
KL divergence for order_proc_id_coded: 3.1035859537292048
************************
************************
************************
Processing column: year
KL divergence for year: 20.806239404724927
************************
************************
************************
************************
Processing column: age
KL divergence for age: 0.012804176866657031
************************
Processing column: gender
KL divergence for gender: 0.00026583946495325333
************************
Processing column: hosp_ward_IP


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/histograms.py:883: RuntimeWarning: divide by zero encountered in divide
  return n/db/n.sum(), bin_edges
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/histograms.py:883: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


KL divergence for hosp_ward_IP: nan
************************
Processing column: hosp_ward_OP
KL divergence for hosp_ward_OP: nan
************************
Processing column: hosp_ward_ER
KL divergence for hosp_ward_ER: 0.05349167923814832
************************
Processing column: hosp_ward_ICU
KL divergence for hosp_ward_ICU: 0.02734527047242226
************************
Processing column: adi_score
KL divergence for adi_score: 0.00566374375293641
************************
Processing column: Q25_heartrate
KL divergence for Q25_heartrate: 0.0054247813942679595
************************
Processing column: Q75_heartrate
KL divergence for Q75_heartrate: 0.007714867456619189
************************
Processing column: median_heartrate
KL divergence for median_heartrate: 0.007101599200279246
************************
Processing column: Q25_resprate
KL divergence for Q25_resprate: 0.01521941499930507
************************
Processing column: Q75_resprate
KL divergence for Q75_resprate: 0.01869

In [13]:
kl_df[kl_df['KL_Divergence'] > 0.1]  # Threshold can be adjusted based on domain knowledge

,KL_Divergence
year,20.806239
order_proc_id_coded,3.103586
nursing_visits_within_6mo,0.773671
STAPHYLOCOCCUS AUREUS_Ciprofloxacin,0.219353
